In [ ]:
import pandas as pd
import altair as alt

df = pd.read_csv("https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/licenses_fall2022.csv")
df['License Type'].nunique()

In [ ]:
df.columns

In [ ]:
state_to_region = {
    # Pacific
    'CA': 'Pacific', 'OR': 'Pacific', 'WA': 'Pacific',

    # West - Actually I mean the Inland West because they are geographically and culturally different from the Pacific west coast 
    'AZ': 'West', 'CO': 'West', 'ID': 'West', 'MT': 'West', 'NV': 'West',
    'NM': 'West', 'UT': 'West', 'WY': 'West',

    # Northeast
    'CT': 'Northeast', 'ME': 'Northeast', 'MA': 'Northeast', 'NH': 'Northeast',
    'RI': 'Northeast', 'VT': 'Northeast', 'NJ': 'Northeast', 'NY': 'Northeast', 'PA': 'Northeast',

    # Midwest
    'IL': 'Midwest', 'IN': 'Midwest', 'IA': 'Midwest', 'KS': 'Midwest', 'MI': 'Midwest',
    'MN': 'Midwest', 'MO': 'Midwest', 'NE': 'Midwest', 'ND': 'Midwest', 'OH': 'Midwest',
    'SD': 'Midwest', 'WI': 'Midwest',

    # South
    'AL': 'South', 'AR': 'South', 'DE': 'South', 'DC': 'South', 'FL': 'South',
    'GA': 'South', 'KY': 'South', 'LA': 'South', 'MD': 'South', 'MS': 'South',
    'NC': 'South', 'OK': 'South', 'SC': 'South', 'TN': 'South', 'TX': 'South',
    'VA': 'South', 'WV': 'South',

    # Alaska and Hawaii
    'AK': 'Alaska-Hawaii',
    'HI': 'Alaska-Hawaii'
}
df['Region'] = df['State'].map(state_to_region)

In [ ]:
df['Region'].value_counts()

In [ ]:
df_clean = df.dropna(subset=['Region', 'License Type', 'State'])

In [ ]:
#From https://altair-viz.github.io/gallery/interactive_legend.html
import altair as alt
alt.data_transformers.disable_max_rows()

selection = alt.selection_point(fields=['License Type'], bind='legend')

alt.Chart(df_clean).mark_bar().encode(
    x=alt.X('Region:N', title='Region'),
    y=alt.Y('count():Q', title='Number of Licenses'),
    color=alt.Color('License Type:N').scale(scheme='category20b'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_params(
    selection
).properties(
    title='License Type Distribution by Region'
)

In [ ]:
midwest_clean = df_clean[df_clean['Region'] == 'Midwest']

grouped = midwest_clean.groupby(['State', 'License Type']).size().reset_index(name='count')

top_10_midwest = (grouped.groupby('License Type')['count'].sum().sort_values(ascending=False).head(10).index.tolist())

grouped = grouped[grouped['License Type'].isin(top_10_midwest)]

state_totals = grouped.groupby('State')['count'].sum().reset_index(name='state_total')

grouped = grouped.merge(state_totals, on='State')

grouped['percent'] = (grouped['count'] / grouped['state_total']) * 100

selection2 = alt.selection_point(fields=['License Type'], bind='legend')

alt.Chart(grouped).mark_area().encode(
    x=alt.X('State:N', title='Midwest States'),
    y=alt.Y('percent:Q', stack='center', title='Percent of Licenses'),
    color=alt.Color('License Type:N').scale(scheme='category20b'),
    opacity=alt.condition(selection2, alt.value(1), alt.value(0.2))
).add_params(
    selection2
).properties(
    title='Top 10 License Types in Midwest States (By % of State total)'
)